In [1]:
# Importing Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats
from scipy.stats import chi2
from scipy.stats import f
import statsmodels.api as sm
import pylab
import statsmodels
from statsmodels import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns

from sklearn.metrics import r2_score

import random 
from random import sample

In [2]:
## Importing the data 
# Importing the data 

datapath = 'D:\Work\Research\Box Sync\Salary History Disclosure\Datawork\Data\Final'
filepath = os.path.join(datapath, 'CPS_ORG_201001201911.dta')
data = pd.read_stata(filepath)

In [3]:
## Check the shape of the data, data types, number of missing values, and dropping of missing values
## Convert Data Types, impute missing values with means within groups, 

#print(data.shape)
#print(data.describe())
#print(data.columns, len(data.columns))
#print(data.dtypes)
#print(data[data.columns].isnull().sum())
#for item in data.columns:
#    if(str(data[item].dtypes)=='category'):
#        print(data[item].value_counts(dropna = False))

## Now convert some of the main data types
data['Age'] = pd.to_numeric(data['Age'], downcast = 'float')
data['year'] = data['year'].astype('category')

# Dropping a variable
data.drop(['asecflag'], axis = 1, inplace = True)

## Now check the number of NaN values in some of the variables and first drop all the NaN values
newdata = data.dropna(axis = 0, how = 'any', inplace = False)

## Now create a smaller data frame which is randomly sampled from a larger data frame 
smallersample = data.sample(frac = 0.7)
print(data.shape, smallersample.shape)
del smallersample
del newdata

(1436525, 48) (1005567, 48)


In [9]:
## Replace NaN values with - specific value, mean, median, mean within some group
print(data.isnull().sum())

## Replace NaN values with specific values
specval = pd.Series([2.0, 35], index = ['ln_weekearn','hoursworked1'])
newdata = data.fillna(specval)
print(data.isnull().sum(), newdata.isnull().sum())
del newdata

## Replace NaN values with one specific value for all
# Checking the data type of the variables which have NaN values
for item in data.columns:
    if(data[item].isnull().sum()>0):
        print(item, data[item].dtypes)
#newdata = data.fillna(0.)
#print(data.isnull().sum(), newdata.isnull().sum())

## Replace NaN values with means of the variable
newdata = data.fillna(data.mean(axis = 0, numeric_only = True, skipna = True))
print(data.isnull().sum(), newdata.isnull().sum())
del newdata

## Replace NaN values with means of the variable within groups
newdata = data.groupby(['sex','educ2','statecensus']).transform(lambda x: x.fillna(x.mean(skipna = True)))
print(data.isnull().sum(), newdata.isnull().sum())
del newdata

year                 0
serial               0
month                0
hwtfinl              0
cpsid                0
mish                 0
pernum               0
wtfinl               0
cpsidp               0
marst                0
famsize              0
nchild               0
citizen              0
multjob              0
numjob               0
schlcoll             0
diffany              0
union                0
metro                0
metarea              0
county               0
statecensus          0
metfips              0
individcc            0
hrhhid               0
hrhhid2              0
Age                  0
sex                  0
empstat              0
classwkr             0
wkstat               0
earnwt               0
lineno               0
hourwage        627733
paidhour             0
earnweek         49849
sector               0
race                 0
educ2                0
industry             0
occupation           0
ptft                 0
AgeSq                0
AgeCu      

In [12]:
## Cross Tabulation of Data values, especially in fractions with total number of observations
pd.crosstab(index = data['sex'], columns = data['educ2'], dropna = False, normalize = 'index', margins = True)

educ2,NoHS,HS,SomeColl,AssocDeg,CollGrad
sex,,,,,
Male,0.080119,0.292199,0.169541,0.102284,0.355856
Female,0.051199,0.237488,0.174500,0.129440,0.407373
All,0.065857,0.265218,0.171987,0.115676,0.381262


In [22]:
## Summarizing Variables by some group measure
data.groupby(['sex','educ2'])['ln_weekearn','hoursworked1'].describe()

C:\Users\Public\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


ln_weekearn                                                   \
                      count      mean       std      min       25%       50%   
sex    educ2                                                                   
Male   NoHS         57959.0  6.230716  0.608426 -4.60517  5.950643  6.214608   
       HS          210184.0  6.537072  0.635808 -4.60517  6.194406  6.579251   
       SomeColl    121211.0  6.570123  0.699898 -4.60517  6.207483  6.645091   
       AssocDeg     73121.0  6.714256  0.628033 -4.60517  6.396930  6.776416   
       CollGrad    229594.0  7.005853  0.686195 -4.60517  6.684612  7.099639   
Female NoHS         36146.0  5.843323  0.610050 -4.60517  5.590278  5.900171   
       HS          167470.0  6.163084  0.663910 -4.60517  5.857933  6.214608   
       SomeColl    122795.0  6.220212  0.705943 -4.60517  5.886104  6.298949   
       AssocDeg     91099.0  6.369328  0.684505 -4.60517  6.036438  6.422240   
       CollGrad    277097.0  6.744320  0.730673 -4.60517  6.422240  6.852401   

                                    hoursworked1                            \
                      75%       max        count       mean       std  min   
sex    educ2                                                                 
Male   NoHS      6.579251  7.966934      54015.0  40.246838  8.599241  0.0   
       HS        6.945051  7.967107     199679.0  41.282333  8.780043  0.0   
       SomeColl  7.050123  7.966934     116054.0  40.792347  9.741561  0.0   
       AssocDeg  7.130899  7.967107      70678.0  41.698647  8.670222  0.0   
       CollGrad  7.456316  7.967138     247344.0  42.926506  9.052106  0.0   
Female NoHS      6.199494  7.882315      33590.0  35.353050  9.502108  0.0   
       HS        6.567418  7.966934     159226.0  36.732368  8.763226  0.0   
       SomeColl  6.659294  7.966934     117040.0  36.442429  9.411265  0.0   
       AssocDeg  6.802395  7.966934      87414.0  36.890797  8.771887  0.0   
       CollGrad  7.205004  7.967107     276588.0  38.989788  9.554200  0.0   

                                         
                  25%   50%   75%   max  
sex    educ2                             
Male   NoHS      40.0  40.0  40.0  99.0  
       HS        40.0  40.0  40.0  99.0  
       SomeColl  40.0  40.0  40.0  99.0  
       AssocDeg  40.0  40.0  40.0  99.0  
       CollGrad  40.0  40.0  45.0  99.0  
Female NoHS      30.0  40.0  40.0  99.0  
       HS        35.0  40.0  40.0  99.0  
       SomeColl  35.0  40.0  40.0  99.0  
       AssocDeg  35.0  40.0  40.0  99.0  
       CollGrad  40.0  40.0  40.0  99.0

In [36]:
## Subsetting Data
## IMPORTANT: loc takes the label and spits out the value, while iloc takes the location and spits out the value
print(data[data.sex == 'Male'].shape)
print(data.loc[data.sex=='Male',:].shape)
print(data[(data.sex=='Male') & (data.Age <= 40.0)].shape)
print(data[(data.sex=='Male') & (data.Age <= 40.0) & (np.isnan(data.ln_weekearn))].shape)

(728089, 48)
(728089, 48)
(340933, 48)
(10104, 48)


In [53]:
## Adding a new column to a data frame
data['Treat'] = 0
data['year'] = pd.to_numeric(data['year'], downcast = 'float')
#print(data['year'].value_counts())
data.loc[data.year > 2015.,['Treat']] = 1
pd.crosstab(data.year, data.Treat)

Treat,0,1
year,,
2010.0,149519,0
2011.0,147811,0
2012.0,147233,0
2013.0,146853,0
2014.0,147609,0
2015.0,145620,0
2016.0,0,145572
2017.0,0,143392
2018.0,0,139590


In [56]:
## Now sorting rows or columns based on values 
newdata = data.sort_values(by = ['ln_weekearn', 'Age'], axis = 0, ascending = [True, False], na_position = 'last')
newdata.head(n=5)

,year,serial,month,hwtfinl,cpsid,mish,pernum,wtfinl,cpsidp,marst,...,industry,occupation,ptft,AgeSq,AgeCu,clustervar,ln_weekearn,hoursworked1,ln_wagerate1,Treat
841994,2015.0,40917,april,1576.0674,2.015010e+13,four,1,2173.7190,2.015010e+13,"married, spouse present",...,Mining,53.0,Full-Time,4096.0,262144.0,882015.0,-4.60517,45.0,2.639057,0
1427742,2019.0,44475,october,3342.6369,2.019070e+13,four,1,3342.6369,2.019070e+13,divorced,...,Education,25.0,Full-Time,4096.0,262144.0,212019.0,-4.60517,40.0,NaN,1
88784,2011.0,36144,february,1724.8317,2.009111e+13,eight,1,1724.8317,2.009111e+13,widowed,...,Professional,11.0,Full-Time,3969.0,250047.0,642011.0,-4.60517,55.0,NaN,0
311549,2012.0,55023,august,2854.1190,2.011051e+13,eight,2,2981.8228,2.011051e+13,"married, spouse present",...,Manufacturing,11.0,Full-Time,3969.0,250047.0,232012.0,-4.60517,50.0,NaN,0
347206,2011.0,46397,november,3203.1010,2.011080e+13,four,1,3328.7676,2.011080e+13,"married, spouse present",...,Others,11.0,Full-Time,3844.0,238328.0,212011.0,-4.60517,40.0,NaN,0


In [83]:
## Now compute the number of duplicate observations within each group and assign a duplicate id to each observation
data.groupby(['statecensus','educ2'])['cpsidp'].count().reset_index()['cpsidp'].unique()

array([  620,  5619,  2915,  2634,  6729,   907,  7510,  4133,  3327,
       11590,   630,  5981,  2778,  2330,  8971,  1122,  6086,  3241,
        2553, 14390,  1346,  4840,  2947,  2192,  7766,  1085,  6219,
        3551,  2401, 11531,  4101, 14784,  7891,  6690, 26773,  1673,
        7707,  4115,  2572, 14328,  1815, 13899,  5317,  5008, 15992,
        1581, 12636,  6574,  4435, 12711,  1371,  7606,  4029,  2701,
        7437,  2666, 10589,  7645,  4432, 18560,  1258,  8521,  6435,
        3870, 11705,  1061,  7186,  4448,  4044,  9088,  1129,  6126,
        4778,  5303, 12359,  1114,  6502,  3975,  3990,  8064,  1067,
        6402,  4280,  2636,  7785,   659,  5473,  3907,  4012,  7918,
         863,  5490,  3149,  3667,  6476,  1443,  5198,  3995,  2910,
        7923,  1203,  4634,  3698,  2429,  7925,  1303,  5973,  3120,
        1875,  7103,  1600,  6786,  4641,  2332, 13526,  1283,  2877,
        1995,   681, 19250,  1757,  7344,  4953,  2723, 13937,  1148,
        7764,  3217,

In [88]:
## Now Assign a unique ID by grouping of variabless
data['GroupID'] = data.groupby(['sex','educ2']).ngroup()
data['GroupID'].unique()

array([2, 9, 4, 7, 6, 1, 0, 3, 8, 5], dtype=int64)

In [104]:
# Drop all duplicated values
newdata = data[~data.reset_index().duplicated(subset=['cpsidp']).values]
print(newdata.shape)
print(data.groupby(['cpsidp'])['Age'].count().reset_index()['Age'].value_counts())
del newdata

(995281, 50)
1    554037
2    441244
Name: Age, dtype: int64


In [5]:
## Now assign a unique duplicate ID to all values within a group sorted by another variable
## CAUTION: there should not be duplicates
newdata = data.dropna(axis = 0, how = 'any')

newdata.sort_values(by = ['cpsidp','mish'], inplace = True)
newdata.reset_index(drop = True, inplace = True)
varlist = newdata.set_index(['cpsidp','mish']).groupby(level = 'cpsidp').cumcount().reset_index().iloc[:,2]
newdata['Rank'] = varlist
pd.crosstab(newdata.mish, newdata.Rank)

C:\Users\Public\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Public\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Rank,0,1
mish,,
four,379775,0
eight,206649,173159


In [94]:
## Now Sort values by some metric within some group, and then create new values as function of previous value

# First sort the values in the order in which we would want them to be 
newdata.sort_values(by = ['cpsidp','mish'], inplace = True)
newdata.reset_index(drop = True, inplace = True)

# Use the shift function 
newdata['AvgEarn'] = np.where(newdata.ln_weekearn.shift(periods = 1, axis = 0)==newdata.ln_weekearn, 
                              (newdata.ln_weekearn.shift(periods = 1, axis = 0)+newdata.ln_weekearn)/2.,newdata.ln_weekearn)
newdata.loc[newdata['Duplicates'], ['cpsidp','mish','ln_weekearn','AvgEarn']]

(759583, 48) 51


In [114]:
# Reshaping Data

#newdata = data.dropna(axis = 0, how = 'any')
#smalldata = newdata[['cpsidp','mish','ln_weekearn','sex','educ2','year','month']]
#print(newdata.shape, smalldata.shape)

#del newdata
# Convert the data from long to wide format
# Works only for single columns at once and they need to be concatenated with the main dataframe 
#new = pd.pivot_table(smalldata, index = ['cpsidp'], columns = ['mish'], 
#                            values = ['ln_weekearn','month'], 
#                           aggfunc = 'first')
#print(new.shape)
#new.columns = [''.join(col) for col in new.columns]
#new = new.reset_index()
#print(new)

In [132]:
newframe = pd.DataFrame({'ID':['A','B','B','A','A'], 
                         'Year':[1,2,1,3,2], 
                         'Revenue':[12.5,13.5,90.,21.,5.], 
                         'Gender':['M','F','F','M','M']})
print(newframe)
new = pd.pivot_table(newframe, index = 'ID', columns = 'Year', values = ['Revenue','Gender'], aggfunc = 'first')
new.columns = [col1+str(col2) for (col1,col2) in new.columns]
print(new.columns)
new = new.reset_index()
print(new)

  ID  Year  Revenue Gender
0  A     1     12.5      M
1  B     2     13.5      F
2  B     1     90.0      F
3  A     3     21.0      M
4  A     2      5.0      M
Index(['Gender1', 'Gender2', 'Gender3', 'Revenue1', 'Revenue2', 'Revenue3'], dtype='object')
  ID Gender1 Gender2 Gender3  Revenue1  Revenue2  Revenue3
0  A       M       M       M      12.5       5.0      21.0
1  B       F       F     NaN      90.0      13.5       NaN


In [134]:
## Converting a data from wide to long format
new = pd.wide_to_long(new, stubnames = ['Gender','Revenue'], i = 'ID', j = 'Year')
print(new)

        Gender  Revenue
ID Year                
A  1         M     12.5
B  1         F     90.0
A  2         M      5.0
B  2         F     13.5
A  3         M     21.0
B  3       NaN      NaN


In [137]:
## Appending data frames

data1 = data.sample(frac = 0.5)
data2 = data.sample(frac = 0.1)
print(data1.shape, data2.shape)
newdata = data1.append(data2)
print(newdata.shape)
del [data1, data2, newdata]

(718262, 48) (143652, 48)
(861914, 48)


In [146]:
## Merging Datasets
data1 = data.sample(frac=0.5)[['Age','cpsidp','mish','ln_weekearn']]
data2 = data.drop(['Age','ln_weekearn'], axis = 1)
print(data1.shape, data2.shape)

newdata = data1.merge(data2, how = 'right', on = ['cpsidp','mish'])
print(newdata.shape)
newdata = data1.merge(data2, how = 'inner', on = ['cpsidp','mish'])
print(newdata.shape)
newdata = data1.merge(data2, how = 'outer', on = ['cpsidp','mish'])
print(newdata.shape)

(718262, 4) (1436525, 46)
(1436525, 48)
(718262, 48)
(1436525, 48)
